Approximation
=============

Scroll down and you should find some approximations of the obfuscation code.
I have no idea what evil forces compelled me to do this, but eh let's roll with it.

In [3]:
using TaylorModels

GCJ-02
------

The `abs()` makes it impossible to feed into taylor, so I think I will just feed the trigs in. Splitting the x and y too, so we don't get a lot of zero in the coeffs.

Let's start with... typing the formulae:

In [4]:
# Not gonna type these pi.
spi(x) = sin(x * pi)

# Just the core meter-level shite. 1 and 2 are to be fed to taylor.
n0(x, y) = -100 + 0.1x*y + 0.2 * sqrt(abs(x))
n1(x) = 2x + 20 / 3 * (2 * spi(6x) + 2 * spi(2x))
n2(y) = 3y + 0.2y^2 + 20 / 3 * (2 * spi(y) + 4 * spi(y/3) + 16 * spi(y/12) + 32 * spi(y/30))

e0(x, y) = 300 + 0.1x*y + 0.1 * sqrt(abs(x)) + 2y
e1(x) = x + 0.1x^2 + 20 / 3 * (2 * spi(6x) + 2 * spi(2x) + 2 * spi(x) + 4 * spi(x/3) + 15 * spi(x/12) + 30 * spi(x/30))

e1 (generic function with 1 method)

Set up for TaylorModels:

In [5]:
x_dom = -30..30
y_dom = -30..20

# 35,105 as the center of China. What can go wrong?
x0 = 0
y0 = 0

ORDER = 6

tmx = TaylorModel1(ORDER, interval(x0), x_dom)
tmy = TaylorModel1(ORDER, interval(y0), y_dom)

n1tm = n1(tmx)
n2tm = n2(tmy)
e1tm = e1(tmx)

println("n1tm = ", n1tm)
println("n2tm = ", n2tm)
println("e1tm = ", e1tm)

n1tm =  [337.103, 337.104] t + [-15434.3, -15434.2] t³ + [265489, 265490] t⁵ + [-4.894e+16, 4.894e+16]
n2tm =  [123.078, 123.079] t + [0.2, 0.200001] t² + [-74.3666, -74.3665] t³ + [34.2831, 34.2832] t⁵ + [-1.74906e+11, 1.74906e+11]
e1tm =  [453.04, 453.041] t + [0.1, 0.100001] t² + [-15508.6, -15508.5] t³ + [265523, 265524] t⁵ + [-4.89402e+16, 4.89402e+16]


Oh well, the last term is way better than what I expected. Let's export them as usual polynomials then.

In [6]:
function mid_tm(t::TaylorModel1)
    coeffs = map(x -> mid(x), t.pol.coeffs)
    order = t.pol.order
    return Taylor1(coeffs, order)
end
n1t = mid_tm(n1tm)
n2t = mid_tm(n2tm)
e1t = mid_tm(e1tm)
println("n1t = ", n1t)
println("n2t = ", n2t)
println("e1t = ", e1t)

n1t = 337.10321638291134 t - 15434.235503082582 t³ + 265489.0776448309 t⁵ + 𝒪(t⁷)
n2t =  123.07865253720988 t + 0.2 t² - 74.36657685107961 t³ + 34.28315615652991 t⁵ + 𝒪(t⁷)
e1t =  453.0402762665314 t + 0.1 t² - 15508.57959081323 t³ + 265523.36073126463 t⁵ + 𝒪(t⁷)


These should be good enough to just translate into code. If you are allowing the compiler to reassoc, there should be no need of manually writing Horner and thinking about pipelines. The reduced number of operations might even make `f32` acceptable. I will eventually put it in the cpp thing and check it.

The arclen stuff have the same `sqrt` issue, so it's not getting changed here.

BD-09
----
We have a `sqrt` in `hypot`, so no. We can't fuze any operations here anyways.


Plotting session
----------------
Oops UI can't install `Plots`. And I just realized the resulting numbers are
unreasonably large. Oops.